### Import dependencies

In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
# import helpers
import numpy as np
import random
import os

# Import stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


### 2. Types of spaces

In [2]:
Discrete(3).sample()

2

In [3]:
Box(0,2, shape=(3,)).sample()

array([0.54688555, 1.5363915 , 1.4802473 ], dtype=float32)

In [4]:
Tuple((Discrete(3),Box(0,1,shape=(3,)))).sample()

(2, array([0.81409514, 0.08350611, 0.49947587], dtype=float32))

In [5]:
Dict({'height':Discrete(2), 'speed': Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([36.782597], dtype=float32))])

In [6]:
MultiBinary(4).sample()

array([1, 0, 0, 0], dtype=int8)

In [7]:
MultiDiscrete([5,2,2]).sample()

array([3, 1, 0], dtype=int64)

### 3. Building and Environment

* Build an agent to give us the best shower possible
* Randomly temperature
* 39 to 39 degrees                                                                                                                              

In [8]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
    def step(self,action):
        # Apply temp adj
        self.state += action-1
        # Decrease shower time
        self.shower_length -= 1
        # Calculate reward
        if self.state >= 37 and self.state <=39:
            reward = 1
        else:
            reward = -1
        if self.shower_length <=0:
            done = True
        else:
            done = False
        info = {}
        return self.state, reward, done, info 
            
    def render(self):
        # Implement viz
        pass
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [9]:
env = ShowerEnv()

In [10]:
env.observation_space.sample()

array([40.935364], dtype=float32)

In [11]:
env.action_space.sample()

1

In [12]:
env.reset()

array([38.])

In [13]:
new_action = env.action_space.sample()
env.step(new_action)

(array([39.]), 1, False, {})

### 4. Test environment

In [14]:
episodes = 5
for episode in range(1,episodes+1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-38
Episode:2 Score:-60
Episode:3 Score:-36
Episode:4 Score:-60
Episode:5 Score:-60


### 5. Train model

In [15]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


d:\RL_Examples_Renotte\venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [16]:
model.learn(total_timesteps=40000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -24.1    |
| time/              |          |
|    fps             | 111      |
|    iterations      | 1        |
|    time_elapsed    | 18       |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 60        |
|    ep_rew_mean          | -26.4     |
| time/                   |           |
|    fps                  | 150       |
|    iterations           | 2         |
|    time_elapsed         | 27        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0107293 |
|    clip_fraction        | 0.105     |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.09     |
|    explained_variance   | -5.82e-05 |
|    learning_rate        | 0.0003    |
|    loss           

### 6. Save Model

In [17]:
shower_path = os.path.join('Training','Saved Models', 'Shower_Model_PPO')
model.save(shower_path)

In [18]:
del model

In [19]:
model = PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
evaluate_policy(model,env,n_eval_episodes=100, render=True)

d:\RL_Examples_Renotte\venv\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
d:\RL_Examples_Renotte\venv\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(-6.0, 59.6992462263972)